In [1]:
%config Completer.use_jedi = False

In [2]:
import os
from skimage import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import albumentations
import torch

from pathlib import Path

from utils.general import *
import utils.dataload as d
from models import model_selector
from utils.data_augmentation import data_augmentation_selector
from medpy.metric.binary import hd, dc, jc, assd

In [3]:
def find_path(directory, filename):
    for path in Path(directory).rglob(filename):
        return path

In [8]:
lvsc_model = model_selector(
    "segmentation", "resnet34_unet_imagenet_encoder_scse_hypercols", 1, from_swa=True,
    in_channels=3, devices=[0], checkpoint="checks/LVSC/n1_100_swa.pt"
)

lvsc_model.eval()
print("LVSC model loaded")


--- Frosted pretrained backbone! ---
Model total number of parameters: 35740845
Loaded model from checkpoint: checks/LVSC/n1_100_swa.pt
LVSC model loaded


In [11]:
acdc_model = model_selector(
    "segmentation", "resnet34_unet_imagenet_encoder_scse_hypercols", 4, from_swa=True,
    in_channels=3, devices=[0], checkpoint="checks/ACDC/n1_100_swa.pt"
)

acdc_model.eval()
print("ACDC model loaded")


--- Frosted pretrained backbone! ---
Model total number of parameters: 35749488
Loaded model from checkpoint: checks/ACDC/n1_100_swa.pt
ACDC model loaded


In [12]:
acdc_model = model_selector(
    "segmentation", "resnet34_unet_imagenet_encoder_scse_hypercols", 4, from_swa=True,
    in_channels=3, devices=[0], checkpoint="checks/MMs/n1_100_swa.pt"
)

acdc_model.eval()
print("MMs model loaded")


--- Frosted pretrained backbone! ---
Model total number of parameters: 35749488
Loaded model from checkpoint: checks/MMs/n1_100_swa.pt
MMs model loaded


In [13]:
_, _, val_aug = data_augmentation_selector(
    "none", 224, 224, "padd"
)

Using None Data Augmentation
Padding masks!
Padding masks!


In [15]:
value_ranges = [0, 0.25, 0.5, 0.75, 1]
values_desc = ["awful", "average", "good", "excellent"]

for i in range(4):
    print(f"{value_ranges[i]} - {value_ranges[i+1]}: {values_desc[i]}")

0 - 0.25: awful
0.25 - 0.5: average
0.5 - 0.75: good
0.75 - 1: excellent


In [19]:
sample = np.random.randint(0,2,(100,100))
sample.shape

(100, 100)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(17, 10))
fig.tight_layout(pad=3)  # Set spacing between plots

ax1.imshow(image, cmap="gray")
ax1.axis("off")
ax1.set_title("Input Image")

masked_lv = np.ma.masked_where(mask == 0, mask)
ax2.imshow(image, cmap="gray")
ax2.imshow(masked_lv, 'hsv', interpolation='bilinear', alpha=0.33)
ax2.axis("off")
ax2.set_title("Ground-truth")

masked_lv = np.ma.masked_where(pred_mask == 0, pred_mask)
ax3.imshow(image, cmap="gray")
ax3.imshow(masked_lv, 'hsv', interpolation='bilinear', alpha=0.33)
ax3.axis("off")
ax3.set_title("Automatic Segmentation")

fig.suptitle(f"{row['ids']} - Jaccard {row['iou']:.4f}", y=0.9)
parent_dir = os.path.join("preds", values_desc[i])
os.makedirs(parent_dir, exist_ok=True)
plt.savefig(os.path.join(parent_dir, f"{row['ids']}.jpg"), dpi=300)
plt.close()

In [16]:
def save_pred(image, mask, pred_mask, row):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(17, 10))
    fig.tight_layout(pad=3)  # Set spacing between plots

    ax1.imshow(image, cmap="gray")
    ax1.axis("off")
    ax1.set_title("Input Image")

    masked_lv = np.ma.masked_where(mask == 0, mask)
    ax2.imshow(image, cmap="gray")
    ax2.imshow(masked_lv, 'hsv', interpolation='bilinear', alpha=0.33)
    ax2.axis("off")
    ax2.set_title("Ground-truth")

    masked_lv = np.ma.masked_where(pred_mask == 0, pred_mask)
    ax3.imshow(image, cmap="gray")
    ax3.imshow(masked_lv, 'hsv', interpolation='bilinear', alpha=0.33)
    ax3.axis("off")
    ax3.set_title("Automatic Segmentation")

    fig.suptitle(f"{row['ids']} - Jaccard {row['iou']:.4f}", y=0.9)
    parent_dir = os.path.join("preds", values_desc[i])
    os.makedirs(parent_dir, exist_ok=True)
    plt.savefig(os.path.join(parent_dir, f"{row['ids']}.jpg"), dpi=300)
    plt.close()

In [15]:
plots_per_range = 25
dicom_dir = "data/LVSC/Validation/"
consensus_dir = "data/LVSC/consensus/images/"

for i in range(4):
    selection = df.loc[(df["iou"]>value_ranges[i]) & (df["iou"]<value_ranges[i+1])]
    current_plots = 0
    for index, row in selection.iterrows():
        patient = row['ids'].split("_")[0]
        
        img_path = os.path.join(find_path(dicom_dir, f"{row['ids']}.dcm"))
        image = d.read_dicom(img_path)
        
        mask_path = os.path.join(consensus_dir, patient, f"{row['ids']}.png")
        mask = np.where(io.imread(mask_path) > 0.5, 1, 0).astype(np.int32)
        
        imageT, maskT = d.apply_augmentations(image, albumentations.Compose(val_aug), [], mask)
        imageT = d.apply_normalization(imageT, "standardize")
        imageT = torch.from_numpy(np.expand_dims(imageT, axis=0))

        imageT = d.add_depth_channels(imageT)
        maskT = torch.from_numpy(np.expand_dims(maskT, 0)).float()
        
        pred_mask = model(imageT.unsqueeze(0)).squeeze()
        pred_mask = reshape_masks(torch.sigmoid(pred_mask).detach().cpu().numpy(),mask.shape, "padd")
        pred_mask = (pred_mask > 0.5, 1, 0)[0].astype(np.int32)
        
        save_pred(image, mask, pred_mask, row)
        
        current_plots += 1
        if current_plots >= plots_per_range:
            break

-------------

## Get metrics by replacing infinite distance values with max value

In [5]:
print(f"Mean IOU: {df['iou'].mean()}")
print(f"Mean DICE: {df['dice'].mean()}")
print(f"Mean Hausdorff: {df['hausdorff'].mean()}")
print(f"Mean ASSD: {df['assd'].mean()}")

Mean IOU: 0.7713022863926522
Mean DICE: 0.8653602617798583
Mean Hausdorff: 3.3798757712666525
Mean ASSD: 0.9405419264204584


In [6]:
max_hausdorff = df["hausdorff"].max()
max_assd = df["assd"].max()

In [7]:
df["hausdorff"].replace(-1, max_hausdorff, inplace=True)
df["assd"].replace(-1, max_assd, inplace=True)

In [8]:
print(f"Mean IOU: {df['iou'].mean()}")
print(f"Mean DICE: {df['dice'].mean()}")
print(f"Mean Hausdorff: {df['hausdorff'].mean()}")
print(f"Mean ASSD: {df['assd'].mean()}")

Mean IOU: 0.7713022863926522
Mean DICE: 0.8653602617798583
Mean Hausdorff: 3.5313756697819394
Mean ASSD: 1.0044842124418878
